# Energy efficient parallel programming

**Note:** this notebook is meant to be executed on the supplied Docker image.

TO DO - EN

Celem zajęć jest zaznajomienie się z technikami programowania wydajnego energetycznie oraz porównanie czasu wykonania z ilością zużytej energii.

In [1]:
import os
from pathlib import Path
import time

from multiprocessing import Pool
from collections import defaultdict, deque
import random


## RAPL sysfs Interface


First of all, it's important to find the zone of your CPU in sysfs. Use the knowledge from the previous modules to find the path of thiz zone and complete the code below.

Please verify your path to the RAPL module in your PC.


In [2]:
cpu_zone = '/sys/devices/virtual/powercap/intel-rapl/intel-rapl:0/'

Let's verify whether this zone seems like the zone of your CPU.

In [3]:
if os.path.isdir(cpu_zone):
    print('✓ Zone exists')
else:
    print('✗ Zone does not exist!')

name = Path(f'{cpu_zone}/name').read_text().strip()

if name.startswith('package-'):
    print('✓ Its name starts with \'package-\'')
else:
    print('✗ Its zone does not start with \'package-\'!')

if os.path.isfile(f'{cpu_zone}/energy_uj'):
    print('✓ File \'energy_uj\' exists')
else:
    print('✗ File \'energy_uj\' does not exist!')

✓ Zone exists
✓ Its name starts with 'package-'
✓ File 'energy_uj' exists


The energy counter which reports energy consumed by the zone in micro joules is available as a file named `energy_uj`. This file returns the energy consumed from ar arbitrary, but fixed point in time. By calculating the difference between these values at different points in time, we can obtain the energy consumed during the measurrement period.


Let's start with rading the value of `energy_uj`. Complete the following code.

In [4]:
def energy_uj():
    # TO DO - remove implementation
    fp = open(cpu_zone+'energy_uj', 'r')
    return int(fp.read())

print(f'Current energy counter value: {energy_uj()}')

Current energy counter value: 2736700208


Next, we can use the energy counter value read at two different times to calculate energy consumption during the measurement period. Complete the following code. Remembebr that the counter may be reset to zero!


In [5]:
def energy_consumption(energy_uj_start, energy_uj_end):
    # TO DO - remove implementation
    fp = open(cpu_zone+'max_energy_range_uj', 'r')
    max_energy_range_uj = int(fp.read())

    if energy_uj_end < energy_uj_start:
        return max_energy_range_uj-energy_uj_start+energy_uj_end
    return energy_uj_end-energy_uj_start

Let's verify whether the function defined above works. Check the output of the script below.

In [6]:
energy = energy_consumption(10000, 10000)
if energy == 0:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

energy = energy_consumption(0, 10000)
if energy == 10000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

counter_max = int(Path(f'{cpu_zone}/max_energy_range_uj').read_text().strip())
energy = energy_consumption(counter_max - 1000, 1000)
if energy == 2000:
    print('✓ Pass')
else:
    print(f'✗ Fail: {energy}')

✓ Pass
✓ Pass
✓ Pass


Let's try measuring idle CPU energy consumption using the functions `energy_uj()` and `energy_consumption()`. Complete the following code to get the energy consumed during the 5 second measurement period in joules.

In [15]:
measure_time = 5  # seconds

# TO DO - remove implementation
start_energy_uj = energy_uj()
time.sleep(measure_time)
end_energy_uj = energy_uj()

energy_consumed_uj = energy_consumption(start_energy_uj, end_energy_uj)
energy_consumed_j = energy_consumed_uj / 10**6 / measure_time

print(f'Energy consumed: {energy_consumed_j} J')

Energy consumed: 7.4792778 J


The energy consumed should be roughly between 10 and 1000 joules depending on the power of your equipment.

**Note**: If you use the ultrabook with low energy consumprion CPU like [Intel® Core™ i7-1165G7](https://ark.intel.com/content/www/us/en/ark/products/208921/intel-core-i7-1165g7-processor-12m-cache-up-to-4-70-ghz-with-ipu.html) the consumed energy may be below 10 joules

Verify the usage after power capping. Write the function to read power caps and power profile names.

In the following files you should read:
1. `constraint_{i}_power_limit_uw`
2. `constraint_{i}_max_power_uw`
3. `constraint_{i}_time_window_us`
4. `constraint_{i}_name`

**Tip:** your method may be similar to `energy_uj()`
**Note:** you can have more than one profiles


In [12]:
def read_power_limits():
    # TO DO - remove implementation
    i = 0
    while True:
        try:
            fp_name = open(cpu_zone+f'constraint_{i}_name', 'r')
            fp_limit = open(cpu_zone+f'constraint_{i}_power_limit_uw', 'r')
            fp_max = open(cpu_zone+f'constraint_{i}_max_power_uw', 'r')
            fp_time = open(cpu_zone+f'constraint_{i}_time_window_us', 'r')
            print(fp_name.read(), fp_limit.read(), fp_max.read(), fp_time.read())
        except:
            break
        
        i+=1

Verify your implementation:

In [14]:
read_power_limits()

long_term
 200000000
 28000000
 27983872

short_term
 64000000
 0
 2440



Now change the power limits. Implement the methods below. Note the provided power limits are in µW (microWatts) 1W = 1 000 000 µW 

In [29]:
def change_power_limits_uw(
    constraint_num: int,
    new_power_val: int
):
    # TO DO - remove implementation
    # fp = open(cpu_zone+f'constraint_{constraint_num}_power_limit_uw', 'w')
    # fp.write(f"{new_power_val}\n")
    print(f"The constraint_{constraint_num}_power_limit_uw was changed to {new_power_val}\n")
    return
    

Verify if the power was changed.

Be very **careful**!\
Remember old values for changed constraint!\
**Do not decrease** the value too much!

In [19]:
constraint_num = None
new_power_val = None

read_power_limits()
change_power_limits_uw(constraint_num, new_power_val)
read_power_limits()


long_term
 200000000
 28000000
 27983872

short_term
 64000000
 0
 2440

The constraint_None_power_limit_uw was changed to None

long_term
 200000000
 28000000
 27983872

short_term
 64000000
 0
 2440



## Benchmark for measure efficiency and power consumption based on graph algorithms 


Below there are example graph

In [9]:
class Graph:
    def __init__(self):
        self.graph = defaultdict(list)

    def add_edge(self, u, v):
        self.graph[u].append(v)

    def bfs(self, start):
        visited = set()
        queue = deque([start])
        result = []

        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                result.append(node)
                queue.extend(self.graph[node])

        return result

    def dfs(self, start):
        visited = set()
        stack = [start]
        result = []

        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                result.append(node)
                stack.extend(neighbor for neighbor in reversed(self.graph[node]) if neighbor not in visited)

        return result

def generate_random_graph(num_nodes, num_edges) -> Graph:
    graph = Graph()
    for _ in range(num_edges):
        u = random.randint(1, num_nodes)
        v = random.randint(1, num_nodes)
        graph.add_edge(u, v)
    return graph

def bfs_wrapper(graph: Graph, start_node):
    return graph.bfs(start_node)

def dfs_wrapper(graph: Graph, start_node):
    return graph.dfs(start_node)


Example usage of the algorithms

In [28]:
num_nodes = 10**5
num_edges = 20**5
graph = generate_random_graph(num_nodes, num_edges)

print("start execution")
start_ts = time.time()
start_energy_uj = energy_uj()

start_node = 1
bfs_result = bfs_wrapper(graph, start_node)
dfs_result = dfs_wrapper(graph, start_node)

end_energy_uj = energy_uj()
finish_ts = time.time()
print("finish execution")

print("BFS Result:", bfs_result)
print("DFS Result:", dfs_result)

energy_consumed_uj = energy_consumption(start_energy_uj, end_energy_uj)
energy_consumed_j = energy_consumed_uj / 10**6 / measure_time

print(f'Energy consumed: {energy_consumed_j} J in the {finish_ts-start_ts} s')

start execution
finish execution
BFS Result: [1, 72951, 516, 41454, 97032, 38357, 4302, 84215, 58362, 83208, 12796, 1427, 29154, 13219, 88117, 54412, 64631, 84946, 60359, 64676, 9534, 55775, 1901, 1489, 75487, 63474, 13938, 76083, 27106, 9481, 46137, 59583, 54078, 80439, 99600, 92382, 22855, 11804, 99503, 14897, 96271, 79821, 93653, 33128, 50867, 60473, 22174, 42602, 26089, 87350, 35426, 45925, 9828, 27899, 5247, 73463, 91878, 54583, 32125, 52134, 12006, 47965, 19268, 89752, 33429, 3381, 49630, 77916, 35224, 54187, 27334, 51830, 97396, 77336, 67592, 14458, 50300, 27234, 12357, 1177, 67221, 87733, 2947, 90961, 92409, 59293, 56549, 99814, 54865, 38925, 53951, 50027, 3148, 80784, 17830, 17075, 10917, 8569, 74742, 16571, 47018, 30019, 20760, 65588, 94665, 42043, 32235, 75128, 33324, 25635, 26749, 10753, 26656, 40256, 70058, 51820, 83495, 60794, 78965, 80173, 59038, 90601, 7048, 86659, 82589, 42787, 90127, 10313, 6804, 98019, 20144, 16979, 51982, 98367, 65425, 72340, 43249, 10942, 40581, 79

Based on the example code find the the used energy during calculation BFS and DFS.

Use decreasing power limit (at least 5 different limits). Measure times. What are your thoughts? 

In [31]:
# TO DO - remove implementation
for power_cap in [200000000,200000000]:
    change_power_limits_uw(0, power_cap)

    print("Limits before execution")
    read_power_limits()

    print("start execution")
    start_ts = time.time()
    start_energy_uj = energy_uj()

    start_node = 1
    bfs_result = bfs_wrapper(graph, start_node)
    dfs_result = dfs_wrapper(graph, start_node)

    end_energy_uj = energy_uj()
    finish_ts = time.time()
    print("finish execution")

    print("BFS Result:", bfs_result)
    print("DFS Result:", dfs_result)

    energy_consumed_uj = energy_consumption(start_energy_uj, end_energy_uj)
    energy_consumed_j = energy_consumed_uj / 10**6 / measure_time

    print(f'Energy consumed: {energy_consumed_j} J in the {finish_ts-start_ts} s')


The constraint_0_power_limit_uw was changed to 200000000

Limits before execution
long_term
 200000000
 28000000
 27983872

short_term
 64000000
 0
 2440

start execution
finish execution
BFS Result: [1, 72951, 516, 41454, 97032, 38357, 4302, 84215, 58362, 83208, 12796, 1427, 29154, 13219, 88117, 54412, 64631, 84946, 60359, 64676, 9534, 55775, 1901, 1489, 75487, 63474, 13938, 76083, 27106, 9481, 46137, 59583, 54078, 80439, 99600, 92382, 22855, 11804, 99503, 14897, 96271, 79821, 93653, 33128, 50867, 60473, 22174, 42602, 26089, 87350, 35426, 45925, 9828, 27899, 5247, 73463, 91878, 54583, 32125, 52134, 12006, 47965, 19268, 89752, 33429, 3381, 49630, 77916, 35224, 54187, 27334, 51830, 97396, 77336, 67592, 14458, 50300, 27234, 12357, 1177, 67221, 87733, 2947, 90961, 92409, 59293, 56549, 99814, 54865, 38925, 53951, 50027, 3148, 80784, 17830, 17075, 10917, 8569, 74742, 16571, 47018, 30019, 20760, 65588, 94665, 42043, 32235, 75128, 33324, 25635, 26749, 10753, 26656, 40256, 70058, 51820, 83495,